# Import

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import operator
import pprint
import sklearn
# import community
from sklearn import metrics
from sklearn import cluster
import pandas as pd
import sys
import random

In [2]:
df = pd.read_csv('../data/filtered-infected.csv')
print df.head()
print df.info()

   Source  Target  posting  grade  anal  oral  kiss  infected
0       0       1        0      0     0     0     0         0
1       0      12      216      0     0     0     0         0
2       0      34      223      0     0     0     0         0
3       0      43      245      0     0     0     0         0
4       0      45      252      0     0     0     0         0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31878 entries, 0 to 31877
Data columns (total 8 columns):
Source      31878 non-null int64
Target      31878 non-null int64
posting     31878 non-null int64
grade       31878 non-null int64
anal        31878 non-null int64
oral        31878 non-null int64
kiss        31878 non-null int64
infected    31878 non-null int64
dtypes: int64(8)
memory usage: 1.9 MB
None


In [3]:
female = {}
male = {}

for idx, row in df.iterrows():
    fe = row['Source']
    ma = row['Target']
    if fe in female:
        female[fe] = female[fe] + 1;
    else:
        female[fe] = 1
    
    if ma in male:
        male[ma] = male[ma] + 1;
    else:
        male[ma] = 1

print "Escort", len(female), "Buyer", len(male), "Total", len(female) + len(male)


Escort 3728 Buyer 5420 Total 9148


# Init Data

In [4]:
# 1st column: Female-id
# 2nd column: Male-id
# 3rd column: Date (in days) of posting
# 4th column: Female grade given by the male (Bad: -1 Neutral: 0  Good: +1)
# 5th column: Anal sex with/without condom?  (Yes: +1  No: -1  Information not available: 0)
# 6th column: Oral sex with condom?          (Yes: +1  No: -1  Information not available: 0)
# 7th column: Mouth kiss?                    (Yes: +1  No: -1  Information not available: 0)
# 8th column: infected?                    (Yes: +1  Information not available: 0)

# a new empty graph object
G = nx.Graph()
Females_Full = []
Males_Full = []

def add_node_to_graph(val, num):
    try:
        # Check if value is in graph
        G.node[val]
    except:
        # Add if there is a null pointer
        G.add_nodes_from([val], bipartite=num);
    

# Add data to list and graph
for idx, row in df.iterrows():
    # From
    fr = row['Source']
    Females_Full.append(fr)
    # To
    to = row['Target']
    Males_Full.append(to)
    
    # Attributes
    posting = row['posting']
    grade = row['grade']
    anal = row['anal']
    oral = row['oral']
    kiss = row['kiss']
    infected = row['infected']
    
    # Add Nodes
    add_node_to_graph(fr, 0)
    add_node_to_graph(to, 1)

    # Add edges
    G.add_edges_from([(fr, to)], posting=posting, grade=grade, anal=anal, oral=oral, kiss=kiss, infected=infected)

# Create Giant Component
giant = max(nx.connected_component_subgraphs(G), key=len)

# Define Methods to be used for our Experiments

In [ ]:
def infect(infected):
    newlyInfected = []
    for iter in infected:
        for encounters in G[iter]:
            newlyInfected.append(encounters)
    
    newlyInfected = set(newlyInfected)
    
    for i in newlyInfected:
        infected.add(i)
    print "Newly Infected: ", len(newlyInfected)
    print "Total Infected: ", len(infected)

In [ ]:
def infectWithProb(infected, probability):
    newlyInfected = []
    for iter in infected:
        for encounters in G[iter]:
            if numpy.random.rand() < probability:
                newlyInfected.append(encounters)
                
    newlyInfected = set(newlyInfected)
    
    for i in newlyInfected:
        infected.add(i)
    print "Newly Infected: ", len(newlyInfected)
    print "Total Infected: ", len(infected)

In [ ]:
def infectWithAnalAndProb(infected, prob):
    newlyInfected = []
    for iter in infected:
        for encounters in G[iter]:
            if G[iter].get(encounters).get('anal') == 1:
                if numpy.random.rand() < prob:
                    newlyInfected.append(encounters)
    
    newlyInfected = set(newlyInfected)
    
    for i in newlyInfected:
        infected.add(i)
    print "Newly Infected: ", len(newlyInfected)
    print "Total Infected: ", len(infected)

In [ ]:
def infection(infected, wave):
    print "Wave: ", wave
    if len(infected) > 8200:
        print "Total Infected: ", len(infected)
        
    else:
        infect(infected)
        infection(infected, wave + 1)

In [ ]:
def infectionWithProb(infected, wave, prob, initialNum):
    print "Wave: ", wave
    if len(infected) > 8200:
        print "Total Infected: ", len(infected)
        print
        print "Growth Rate: ", ((len(infected) - initialNum) / wave) /100.00
        
    else:
        infectWithProb(infected, prob)
        infectionWithProb(infected, wave + 1, prob, initialNum)

In [ ]:
def infectionWithAnalAndProb(infected, wave, prob):
    print "Wave: ", wave
    if len(infected) > 8200:
        print "Total Infected: ", len(infected)
        
    else:
        infectWithAnalAndProb(infected, prob)
        infectionWithAnalAndProb(infected, wave + 1, prob)

# Viral Outbreak Tracking

## Create Initially infected Group with HIV

In [10]:
print Females_Full

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 7, 7, 7, 7, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 15, 15, 15, 15, 15, 15, 15, 15, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 1

In [11]:
femaleSet = set(Females_Full)
print femaleSet

set([0, 2, 5, 16390, 7, 9, 11, 16396, 12290, 16398, 15, 17, 18, 8211, 8212, 21, 22, 8215, 24, 8217, 26, 31, 16416, 35, 8228, 37, 39, 41, 16391, 44, 14186, 46, 47, 48, 16433, 8242, 16435, 16436, 16437, 56, 16443, 8252, 61, 63, 16448, 65, 8203, 68, 8261, 71, 73, 16458, 75, 76, 79, 81, 8274, 83, 8276, 85, 16471, 88, 8281, 8207, 8284, 95, 98, 101, 102, 104, 105, 8299, 8300, 16493, 110, 112, 9576, 114, 8307, 16500, 16501, 118, 120, 123, 11512, 126, 8213, 15125, 132, 8214, 135, 8329, 138, 16523, 140, 16525, 142, 8335, 144, 146, 150, 152, 8345, 155, 16543, 15281, 164, 8357, 8358, 168, 15047, 172, 173, 16559, 8368, 8369, 8370, 9587, 8375, 16569, 187, 8385, 16646, 195, 8392, 8396, 205, 8398, 207, 208, 8401, 8227, 16420, 219, 223, 8417, 16611, 9282, 8423, 8424, 233, 234, 16423, 237, 8434, 244, 245, 247, 8443, 256, 9600, 16427, 262, 8455, 265, 8460, 16656, 273, 275, 12557, 285, 8478, 8480, 8481, 293, 8486, 295, 297, 16682, 300, 16434, 11194, 11467, 311, 314, 316, 317, 15559, 319, 320, 322, 324, 3

In [12]:
initiallyInfected = []

for i in femaleSet:
    if numpy.random.rand() <.049:
        initiallyInfected.append(i)
        print(i)

8211
8203
79
8274
8213
314
8530
455
8711
8724
8726
8863
8864
690
8906
8941
8946
797
804
866
9099
9100
956
963
1066
1100
1129
1148
1179
9379
9442
1267
8465
9601
13889
1458
9795
9818
9866
9876
1716
9930
1756
9952
9955
1786
1791
1819
1837
10031
12607
10205
2024
2045
10286
2160
10383
2201
10985
2239
10519
10561
2388
2484
10687
10749
9991
2615
2650
10856
10864
2713
10986
2835
2916
11119
14144
3016
11261
3081
3113
11330
11361
11396
11423
11437
3281
8745
11610
3513
11709
11747
3583
3614
3678
11881
11908
3778
12016
12031
3865
12061
12349
12411
4263
4274
11054
12656
4497
12791
4680
12886
4783
4887
4900
13126
9018
5063
5078
13296
13340
5173
5224
13442
5255
5305
5362
5411
5436
13631
13633
5506
5522
13725
13736
5630
5737
13968
10287
5856
14130
5957
11927
14238
6101
14305
6121
6176
14375
6261
14519
6367
10633
14737
6541
14783
6605
14838
6776
6797
15091
15118
15126
6943
6992
15223
15229
15292
15411
7236
15432
10764
15436
7287
7297
15650
15656
15671
7558
15785
11180
7627
7643
15911
7817
16043
12234
1

In [13]:
print initiallyInfected
print len(initiallyInfected)

[8211, 8203, 79, 8274, 8213, 314, 8530, 455, 8711, 8724, 8726, 8863, 8864, 690, 8906, 8941, 8946, 797, 804, 866, 9099, 9100, 956, 963, 1066, 1100, 1129, 1148, 1179, 9379, 9442, 1267, 8465, 9601, 13889, 1458, 9795, 9818, 9866, 9876, 1716, 9930, 1756, 9952, 9955, 1786, 1791, 1819, 1837, 10031, 12607, 10205, 2024, 2045, 10286, 2160, 10383, 2201, 10985, 2239, 10519, 10561, 2388, 2484, 10687, 10749, 9991, 2615, 2650, 10856, 10864, 2713, 10986, 2835, 2916, 11119, 14144, 3016, 11261, 3081, 3113, 11330, 11361, 11396, 11423, 11437, 3281, 8745, 11610, 3513, 11709, 11747, 3583, 3614, 3678, 11881, 11908, 3778, 12016, 12031, 3865, 12061, 12349, 12411, 4263, 4274, 11054, 12656, 4497, 12791, 4680, 12886, 4783, 4887, 4900, 13126, 9018, 5063, 5078, 13296, 13340, 5173, 5224, 13442, 5255, 5305, 5362, 5411, 5436, 13631, 13633, 5506, 5522, 13725, 13736, 5630, 5737, 13968, 10287, 5856, 14130, 5957, 11927, 14238, 6101, 14305, 6121, 6176, 14375, 6261, 14519, 6367, 10633, 14737, 6541, 14783, 6605, 14838, 6776,

In [14]:
for iter in initiallyInfected:
    print G[iter]

{9377: {'infected': 0, 'posting': 1537, 'kiss': -1, 'grade': -1, 'anal': 0, 'oral': -1}, 9349: {'infected': 0, 'posting': 1530, 'kiss': 1, 'grade': -1, 'anal': 0, 'oral': 1}, 11115: {'infected': 0, 'posting': 1726, 'kiss': -1, 'grade': -1, 'anal': -1, 'oral': -1}, 4268: {'infected': 0, 'posting': 1426, 'kiss': -1, 'grade': -1, 'anal': 1, 'oral': -1}, 9940: {'infected': 0, 'posting': 1667, 'kiss': -1, 'grade': -1, 'anal': -1, 'oral': -1}, 4053: {'infected': 0, 'posting': 1725, 'kiss': -1, 'grade': -1, 'anal': 1, 'oral': 1}}
{8512: {'infected': 0, 'posting': 1455, 'kiss': -1, 'grade': 1, 'anal': 1, 'oral': -1}, 6627: {'infected': 0, 'posting': 1434, 'kiss': -1, 'grade': 1, 'anal': 1, 'oral': -1}, 8165: {'infected': 0, 'posting': 1618, 'kiss': -1, 'grade': 1, 'anal': 1, 'oral': -1}, 1521: {'infected': 0, 'posting': 1568, 'kiss': -1, 'grade': 1, 'anal': 0, 'oral': -1}, 4333: {'infected': 0, 'posting': 1571, 'kiss': -1, 'grade': 0, 'anal': 1, 'oral': -1}, 7697: {'infected': 0, 'posting': 14

In [17]:
newlyInfected = []

for iter in initiallyInfected:
    for encounters in G[iter]:
        newlyInfected.append(encounters)

In [22]:
newlyInfected = set(newlyInfected)
len(newlyInfected)

1305

In [26]:
totalInfected = newlyInfected
for i in initiallyInfected:
    totalInfected.add(i)

In [27]:
print len(totalInfected)

1496


In [28]:
newlyInfected = []

for iter in totalInfected:
    for encounters in G[iter]:
        newlyInfected.append(encounters)

In [29]:
newlyInfected = set(newlyInfected)
len(newlyInfected)

4284

In [30]:
for i in newlyInfected:
    totalInfected.add(i)

In [31]:
print len(totalInfected)

4284


In [32]:
print totalInfected

set([0, 2, 16389, 16390, 7, 9, 11, 16396, 16398, 15, 17, 18, 21, 22, 24, 26, 16414, 16415, 16416, 35, 36, 37, 16423, 41, 16391, 44, 46, 47, 48, 16435, 16436, 16437, 54, 56, 58, 16443, 16444, 65, 68, 69, 71, 73, 74, 75, 76, 79, 80, 16465, 83, 16468, 85, 86, 88, 91, 95, 98, 99, 101, 104, 105, 107, 108, 16493, 110, 112, 114, 16500, 16501, 118, 123, 126, 128, 132, 135, 16521, 138, 16523, 140, 16525, 142, 144, 150, 152, 16537, 154, 155, 156, 157, 16543, 161, 162, 16547, 164, 168, 169, 171, 172, 173, 16559, 177, 16563, 181, 182, 16569, 31, 16575, 195, 16586, 207, 16593, 211, 212, 213, 16420, 223, 16611, 16615, 233, 237, 244, 245, 247, 255, 257, 16427, 16646, 265, 268, 270, 16656, 273, 275, 277, 284, 16670, 293, 297, 298, 299, 300, 304, 311, 314, 316, 317, 319, 320, 322, 324, 325, 328, 329, 334, 338, 339, 343, 345, 346, 348, 349, 350, 352, 355, 357, 358, 359, 368, 370, 372, 374, 375, 379, 385, 388, 392, 393, 395, 397, 398, 403, 404, 419, 426, 429, 433, 435, 16457, 440, 446, 449, 450, 451, 455

In [33]:
newlyInfected = []

for iter in totalInfected:
    for encounters in G[iter]:
        newlyInfected.append(encounters)

In [34]:
newlyInfected = set(newlyInfected)
len(newlyInfected)

8280

In [35]:
for i in newlyInfected:
    totalInfected.add(i)

In [36]:
print len(totalInfected)

8280


In [37]:
print totalInfected

set([0, 1, 2, 16389, 16390, 7, 9, 11, 16396, 16398, 15, 16, 17, 18, 19, 21, 22, 24, 26, 16412, 16414, 16415, 16416, 33, 34, 35, 36, 37, 16423, 40, 41, 16391, 44, 45, 46, 47, 48, 50, 16435, 16436, 16437, 54, 55, 56, 57, 58, 16443, 16444, 65, 66, 68, 69, 71, 73, 74, 75, 76, 78, 79, 80, 16465, 82, 83, 16468, 85, 86, 88, 89, 90, 91, 95, 96, 97, 98, 99, 16484, 101, 103, 104, 105, 106, 107, 108, 16493, 110, 111, 112, 114, 16500, 16501, 118, 121, 122, 123, 124, 16509, 126, 127, 128, 129, 16514, 132, 134, 135, 16521, 138, 16523, 140, 16525, 142, 144, 145, 147, 16533, 150, 151, 152, 16537, 154, 155, 156, 157, 16543, 160, 161, 162, 16547, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 16558, 16559, 177, 16563, 181, 182, 184, 16569, 6, 31, 16572, 189, 190, 16575, 32, 195, 196, 198, 200, 201, 16586, 203, 16588, 207, 16592, 16593, 211, 212, 213, 214, 216, 16420, 16602, 220, 222, 223, 225, 16611, 230, 16615, 233, 236, 237, 238, 239, 244, 245, 247, 250, 251, 252, 16638, 255, 257, 16427, 16646, 264

# Now let's pick our starting node as 4569 - has highest degree centrality,  and begin running the methods above

In [39]:
print len(G[4569])

257


In [112]:
infectWithAnalAndProb({4569, 1322}, .33)

Newly Infected:  73
Total Infected:  75


In [56]:
infect({4569, 1})

Newly Infected:  277
Total Infected:  279


In [58]:
infectWithProb({4569, 1}, .6)

Newly Infected:  165
Total Infected:  167


# Running simulations with the recursive Infection Function

In [66]:
infection({4569, 1}, 0)

Wave:  0
Newly Infected:  277
Total Infected:  279
Wave:  1
Newly Infected:  1877
Total Infected:  1877
Wave:  2
Newly Infected:  5560
Total Infected:  5560
Wave:  3
Newly Infected:  8009
Total Infected:  8009
Wave:  4
Newly Infected:  8998
Total Infected:  8998
Wave:  5
Total Infected:  8998


In [128]:
infectionWithProb({4569, 1}, 0, .01)

Wave:  0
Newly Infected:  1
Total Infected:  3
Wave:  1
Newly Infected:  4
Total Infected:  7
Wave:  2
Newly Infected:  5
Total Infected:  12
Wave:  3
Newly Infected:  6
Total Infected:  18
Wave:  4
Newly Infected:  3
Total Infected:  20
Wave:  5
Newly Infected:  10
Total Infected:  30
Wave:  6
Newly Infected:  3
Total Infected:  33
Wave:  7
Newly Infected:  9
Total Infected:  41
Wave:  8
Newly Infected:  12
Total Infected:  51
Wave:  9
Newly Infected:  18
Total Infected:  66
Wave:  10
Newly Infected:  27
Total Infected:  90
Wave:  11
Newly Infected:  23
Total Infected:  112
Wave:  12
Newly Infected:  36
Total Infected:  145
Wave:  13
Newly Infected:  25
Total Infected:  163
Wave:  14
Newly Infected:  34
Total Infected:  195
Wave:  15
Newly Infected:  50
Total Infected:  232
Wave:  16
Newly Infected:  64
Total Infected:  286
Wave:  17
Newly Infected:  74
Total Infected:  344
Wave:  18
Newly Infected:  68
Total Infected:  389
Wave:  19
Newly Infected:  100
Total Infected:  461
Wave:  20

In [185]:
infectionWithProb({4569, 1}, 0, .33, 2)

Wave:  0
Newly Infected:  101
Total Infected:  103
Wave:  1
Newly Infected:  442
Total Infected:  510
Wave:  2
Newly Infected:  2150
Total Infected:  2280
Wave:  3
Newly Infected:  3625
Total Infected:  4107
Wave:  4
Newly Infected:  4870
Total Infected:  5750
Wave:  5
Newly Infected:  5847
Total Infected:  7152
Wave:  6
Newly Infected:  6330
Total Infected:  8021
Wave:  7
Newly Infected:  6581
Total Infected:  8494
Wave:  8
Total Infected:  8494

Growth Rate:  10.61


# Let's test the effect of centrality on viral outbreaks
## Top Centerality (4569, 4585, 1599)
## Bottom Centerality (16656, 5727, 16585)


In [134]:
infect({4569, 4585, 1599})

Newly Infected:  507
Total Infected:  510


In [186]:
infectionWithProb({4569, 4585, 1599}, 0, .33, 3)

Wave:  0
Newly Infected:  180
Total Infected:  183
Wave:  1
Newly Infected:  624
Total Infected:  732
Wave:  2
Newly Infected:  2745
Total Infected:  2911
Wave:  3
Newly Infected:  4200
Total Infected:  4854
Wave:  4
Newly Infected:  5402
Total Infected:  6472
Wave:  5
Newly Infected:  6108
Total Infected:  7611
Wave:  6
Newly Infected:  6446
Total Infected:  8260
Wave:  7
Total Infected:  8260

Growth Rate:  11.79


In [135]:
infect({16656, 5727, 16585})

Newly Infected:  5
Total Infected:  8


In [188]:
infectionWithProb({16656, 5727, 16585}, 0, .33, 3)

Wave:  0
Newly Infected:  1
Total Infected:  4
Wave:  1
Newly Infected:  1
Total Infected:  5
Wave:  2
Newly Infected:  5
Total Infected:  9
Wave:  3
Newly Infected:  22
Total Infected:  27
Wave:  4
Newly Infected:  70
Total Infected:  87
Wave:  5
Newly Infected:  186
Total Infected:  220
Wave:  6
Newly Infected:  358
Total Infected:  422
Wave:  7
Newly Infected:  1066
Total Infected:  1211
Wave:  8
Newly Infected:  2354
Total Infected:  2780
Wave:  9
Newly Infected:  4601
Total Infected:  5198
Wave:  10
Newly Infected:  5661
Total Infected:  6865
Wave:  11
Newly Infected:  6335
Total Infected:  7891
Wave:  12
Newly Infected:  6544
Total Infected:  8458
Wave:  13
Total Infected:  8458

Growth Rate:  6.5


In [106]:
G[1].get(0).get('anal')

0

In [123]:
infectionWithAnalAndProb({4569, 1}, 0, .01)

Wave:  0
Newly Infected:  1
Total Infected:  3
Wave:  1
Newly Infected:  0
Total Infected:  3
Wave:  2
Newly Infected:  4
Total Infected:  7
Wave:  3
Newly Infected:  0
Total Infected:  7
Wave:  4
Newly Infected:  1
Total Infected:  8
Wave:  5
Newly Infected:  6
Total Infected:  14
Wave:  6
Newly Infected:  6
Total Infected:  20
Wave:  7
Newly Infected:  5
Total Infected:  25
Wave:  8
Newly Infected:  6
Total Infected:  30
Wave:  9
Newly Infected:  4
Total Infected:  33
Wave:  10
Newly Infected:  5
Total Infected:  38
Wave:  11
Newly Infected:  13
Total Infected:  49
Wave:  12
Newly Infected:  16
Total Infected:  64
Wave:  13
Newly Infected:  19
Total Infected:  80
Wave:  14
Newly Infected:  17
Total Infected:  94
Wave:  15
Newly Infected:  16
Total Infected:  107
Wave:  16
Newly Infected:  28
Total Infected:  132
Wave:  17
Newly Infected:  24
Total Infected:  155
Wave:  18
Newly Infected:  30
Total Infected:  175
Wave:  19
Newly Infected:  26
Total Infected:  195
Wave:  20
Newly Infec

KeyboardInterrupt: 

# Infection with Different Probablities

In [141]:
infection(set(initiallyInfected), 0)

Wave:  0
Newly Infected:  1305
Total Infected:  1496
Wave:  1
Newly Infected:  4284
Total Infected:  4284
Wave:  2
Newly Infected:  8280
Total Infected:  8280
Wave:  3
Total Infected:  8280


In [173]:
infectionWithProb(set(initiallyInfected), 0, .25)

Wave:  0
Newly Infected:  409
Total Infected:  600
Wave:  1
Newly Infected:  1280
Total Infected:  1651
Wave:  2
Newly Infected:  3518
Total Infected:  4096
Wave:  3
Newly Infected:  4761
Total Infected:  5997
Wave:  4
Newly Infected:  5363
Total Infected:  7160
Wave:  5
Newly Infected:  5493
Total Infected:  7815
Wave:  6
Newly Infected:  5776
Total Infected:  8257
Wave:  7
Total Infected:  8257

Growth Rate:  11.52


In [174]:
infectionWithProb(set(initiallyInfected), 0, .5)

Wave:  0
Newly Infected:  726
Total Infected:  917
Wave:  1
Newly Infected:  2557
Total Infected:  2889
Wave:  2
Newly Infected:  6270
Total Infected:  6669
Wave:  3
Newly Infected:  7314
Total Infected:  8121
Wave:  4
Newly Infected:  7678
Total Infected:  8717
Wave:  5
Total Infected:  8717

Growth Rate:  17.05


In [175]:
infectionWithProb(set(initiallyInfected), 0, .01)

Wave:  0
Newly Infected:  19
Total Infected:  210
Wave:  1
Newly Infected:  19
Total Infected:  229
Wave:  2
Newly Infected:  8
Total Infected:  237
Wave:  3
Newly Infected:  28
Total Infected:  264
Wave:  4
Newly Infected:  24
Total Infected:  286
Wave:  5
Newly Infected:  34
Total Infected:  318
Wave:  6
Newly Infected:  44
Total Infected:  356
Wave:  7
Newly Infected:  45
Total Infected:  399
Wave:  8
Newly Infected:  65
Total Infected:  455
Wave:  9
Newly Infected:  78
Total Infected:  522
Wave:  10
Newly Infected:  94
Total Infected:  593
Wave:  11
Newly Infected:  104
Total Infected:  673
Wave:  12
Newly Infected:  119
Total Infected:  757
Wave:  13
Newly Infected:  144
Total Infected:  867
Wave:  14
Newly Infected:  130
Total Infected:  957
Wave:  15
Newly Infected:  163
Total Infected:  1061
Wave:  16
Newly Infected:  176
Total Infected:  1176
Wave:  17
Newly Infected:  207
Total Infected:  1305
Wave:  18
Newly Infected:  182
Total Infected:  1401
Wave:  19
Newly Infected:  226

# Now let's see how fast HPV spreads,  which already has a prevalance of 67.7%

In [149]:
initiallyInfectedWithHPV = []

for i in femaleSet:
    if numpy.random.rand() <.677:
        initiallyInfectedWithHPV.append(i)

In [155]:
print len(initiallyInfectedWithHPV)
print
print initiallyInfectedWithHPV

2502

[0, 16390, 9, 11, 16396, 16398, 15, 17, 18, 8211, 8212, 22, 24, 26, 16416, 35, 37, 39, 41, 16391, 44, 46, 48, 8242, 16435, 16436, 16437, 56, 61, 16448, 65, 8203, 68, 8261, 71, 73, 16458, 75, 76, 81, 83, 8276, 16471, 8207, 8284, 95, 98, 102, 105, 8299, 8300, 16493, 110, 112, 9576, 8307, 16500, 16501, 120, 123, 11512, 126, 8213, 135, 16523, 140, 16525, 142, 146, 150, 152, 8345, 15281, 8357, 8358, 168, 15047, 172, 173, 8368, 8369, 8370, 9587, 16569, 8385, 16646, 195, 8392, 8398, 207, 8227, 223, 8417, 16611, 8423, 233, 16423, 237, 8434, 245, 247, 256, 9600, 16427, 285, 8478, 8481, 8486, 295, 16682, 16434, 11194, 11467, 311, 15559, 319, 320, 322, 324, 10977, 328, 332, 334, 8530, 339, 8533, 348, 8545, 355, 359, 16379, 368, 8563, 372, 12350, 374, 375, 8569, 8570, 379, 385, 8578, 8579, 388, 8581, 392, 398, 403, 8597, 8603, 8606, 9627, 426, 8631, 446, 449, 450, 455, 8648, 464, 466, 8662, 8663, 473, 8271, 479, 8673, 482, 487, 8681, 493, 497, 8275, 511, 513, 527, 529, 8723, 8724, 533, 8726,

# Chlamydia

In [152]:
initiallyInfectedWithChlamydia = []

for i in femaleSet:
    if numpy.random.rand() <.205:
        initiallyInfectedWithChlamydia.append(i)

In [156]:
print len(initiallyInfectedWithChlamydia)
print
print initiallyInfectedWithChlamydia

770

[0, 8215, 16416, 47, 16443, 75, 79, 104, 8299, 120, 11512, 15125, 8329, 16523, 155, 16543, 15281, 8358, 187, 16420, 8443, 297, 319, 324, 332, 8533, 8563, 12350, 385, 403, 404, 8597, 429, 8632, 446, 13654, 8271, 482, 493, 11005, 8275, 16468, 507, 8736, 12381, 561, 563, 9652, 8765, 598, 603, 8801, 8802, 8804, 625, 8856, 667, 678, 690, 699, 704, 8904, 8918, 729, 738, 8931, 8939, 8946, 761, 769, 795, 16519, 9007, 836, 860, 866, 871, 872, 888, 14487, 898, 9097, 9099, 8344, 16670, 920, 15173, 9124, 970, 984, 9182, 12453, 997, 998, 1002, 9201, 1016, 1017, 1025, 9239, 1048, 9241, 9250, 1072, 16563, 1084, 9278, 9285, 9291, 11106, 1103, 9324, 9368, 9386, 9409, 8400, 9477, 13702, 9497, 9513, 1327, 9521, 9556, 9557, 9561, 11151, 9564, 12521, 11156, 8426, 9626, 9645, 1455, 9653, 9667, 1476, 9683, 9684, 1498, 1510, 9712, 1526, 1531, 1569, 1575, 1594, 1597, 1602, 9802, 9815, 1629, 1630, 1639, 9833, 9834, 9835, 9847, 1679, 1682, 1687, 1691, 1692, 9888, 12571, 9897, 1711, 9930, 1756, 9955, 9963, 1

# Any STD

In [158]:
initiallyInfectedWithSTD = []

for i in femaleSet:
    if numpy.random.rand() <.716:
        initiallyInfectedWithSTD.append(i)

In [159]:
print len(initiallyInfectedWithSTD)
print
print initiallyInfectedWithSTD

2678

[0, 16390, 7, 16396, 12290, 16398, 15, 17, 18, 21, 22, 8215, 26, 31, 16416, 35, 8228, 37, 39, 16391, 14186, 46, 47, 48, 16433, 8242, 16435, 16437, 56, 61, 16448, 65, 8261, 71, 16458, 81, 83, 8276, 16471, 88, 8281, 8284, 98, 101, 102, 104, 105, 8299, 8300, 16493, 110, 112, 8307, 16500, 16501, 118, 120, 126, 8213, 15125, 132, 8214, 135, 8329, 138, 140, 142, 8335, 144, 8345, 155, 16543, 15281, 8358, 168, 15047, 173, 8368, 8369, 8370, 9587, 8375, 187, 8385, 16646, 8392, 8396, 8398, 207, 208, 8401, 8227, 16420, 219, 223, 8417, 9282, 8424, 233, 237, 8434, 244, 245, 247, 8443, 9600, 16427, 262, 8455, 265, 8460, 273, 12557, 285, 8478, 8480, 8481, 8486, 295, 297, 16682, 16434, 11194, 11467, 316, 317, 15559, 319, 320, 324, 325, 10977, 328, 332, 334, 8530, 339, 8533, 343, 348, 8545, 357, 359, 366, 16379, 368, 8563, 12350, 374, 375, 8569, 8570, 379, 8578, 388, 8581, 392, 397, 398, 403, 404, 8603, 8617, 433, 435, 8631, 8634, 446, 449, 450, 455, 8648, 466, 8663, 8664, 479, 8672, 8673, 482, 487

# Experimenting with More Probablities

In [190]:
infectionWithProb(set(initiallyInfectedWithHPV), 0, .33, len(initiallyInfectedWithHPV))

Wave:  0
Newly Infected:  3353
Total Infected:  5855
Wave:  1
Newly Infected:  5906
Total Infected:  7565
Wave:  2
Newly Infected:  6510
Total Infected:  8411
Wave:  3
Total Infected:  8411

Growth Rate:  19.69


In [176]:
infectionWithProb(set(initiallyInfected), 0, .75)

Wave:  0
Newly Infected:  1021
Total Infected:  1212
Wave:  1
Newly Infected:  3492
Total Infected:  3723
Wave:  2
Newly Infected:  7584
Total Infected:  7810
Wave:  3
Newly Infected:  8544
Total Infected:  8824
Wave:  4
Total Infected:  8824

Growth Rate:  21.58


In [177]:
infectionWithProb(set(initiallyInfected), 0, .1)

Wave:  0
Newly Infected:  148
Total Infected:  339
Wave:  1
Newly Infected:  369
Total Infected:  678
Wave:  2
Newly Infected:  937
Total Infected:  1436
Wave:  3
Newly Infected:  1654
Total Infected:  2515
Wave:  4
Newly Infected:  2186
Total Infected:  3631
Wave:  5
Newly Infected:  2681
Total Infected:  4614
Wave:  6
Newly Infected:  2814
Total Infected:  5389
Wave:  7
Newly Infected:  2975
Total Infected:  6034
Wave:  8
Newly Infected:  3113
Total Infected:  6557
Wave:  9
Newly Infected:  3214
Total Infected:  6967
Wave:  10
Newly Infected:  3374
Total Infected:  7346
Wave:  11
Newly Infected:  3332
Total Infected:  7609
Wave:  12
Newly Infected:  3394
Total Infected:  7874
Wave:  13
Newly Infected:  3422
Total Infected:  8069
Wave:  14
Newly Infected:  3421
Total Infected:  8233
Wave:  15
Total Infected:  8233

Growth Rate:  5.36


In [178]:
infectionWithProb(set(initiallyInfected), 0, .33)

Wave:  0
Newly Infected:  506
Total Infected:  697
Wave:  1
Newly Infected:  1668
Total Infected:  2058
Wave:  2
Newly Infected:  4497
Total Infected:  5042
Wave:  3
Newly Infected:  5813
Total Infected:  6916
Wave:  4
Newly Infected:  6322
Total Infected:  7886
Wave:  5
Newly Infected:  6589
Total Infected:  8423
Wave:  6
Total Infected:  8423

Growth Rate:  13.72


In [179]:
infectionWithProb(set(initiallyInfected), 0, .99)

Wave:  0
Newly Infected:  1295
Total Infected:  1486
Wave:  1
Newly Infected:  4262
Total Infected:  4270
Wave:  2
Newly Infected:  8271
Total Infected:  8277
Wave:  3
Total Infected:  8277

Growth Rate:  26.95


In [191]:
infectionWithProb(set(initiallyInfectedWithSTD), 0, .33, len(initiallyInfectedWithSTD))

Wave:  0
Newly Infected:  3500
Total Infected:  6178
Wave:  1
Newly Infected:  6001
Total Infected:  7743
Wave:  2
Newly Infected:  6638
Total Infected:  8490
Wave:  3
Total Infected:  8490

Growth Rate:  19.37


In [192]:
infectionWithProb(set(initiallyInfectedWithChlamydia), 0, .33, len(initiallyInfectedWithChlamydia))

Wave:  0
Newly Infected:  1450
Total Infected:  2220
Wave:  1
Newly Infected:  3344
Total Infected:  4325
Wave:  2
Newly Infected:  5788
Total Infected:  6753
Wave:  3
Newly Infected:  6472
Total Infected:  7961
Wave:  4
Newly Infected:  6611
Total Infected:  8502
Wave:  5
Total Infected:  8502

Growth Rate:  15.46
